In [38]:
import requests
from bs4 import BeautifulSoup
import time
from tqdm import tqdm
import pandas as pd

In [29]:
import requests
from bs4 import BeautifulSoup
import time

def clean_text(soup):
    # Supprimer les balises script et style qui ne contiennent pas de texte pertinent
    for script in soup(["script", "style", "noscript"]):
        script.decompose()
    
    # Extraire le texte de manière générale dans plusieurs balises
    text = soup.get_text(separator="\n")
    
    # Nettoyer le texte pour supprimer les espaces en trop et les lignes vides
    lines = [line.strip() for line in text.splitlines()]
    text = "\n".join(line for line in lines if line)
    
    return text

def google_search_scraper(query, k):
    query = query.replace(' ', '+')
    url = f"https://www.google.com/search?q={query}&num={k}"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    
    # Effectuer la requête
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        print(f"Erreur : Impossible de récupérer les résultats (code {response.status_code})")
        return {}

    # Parser le HTML avec BeautifulSoup
    soup = BeautifulSoup(response.text, "html.parser")

    results = []
    for g in soup.find_all('div', class_='g'):
        link = g.find('a')['href']
        results.append(link)
        if len(results) >= k:
            break
    
    content_dict = {}
    for site in results:
        try:
            # Récupérer le contenu du site
            site_response = requests.get(site, headers=headers)
            if site_response.status_code != 200:
                print(f"Erreur lors de la récupération du site {site} (code {site_response.status_code})")
                continue
            
            site_soup = BeautifulSoup(site_response.text, "html.parser")
            
            # Nettoyage et extraction de tout le texte
            site_text = clean_text(site_soup)
            
            # Si le texte extrait est trop court, essayer d'autres méthodes
            if len(site_text) < 500:
                paragraphs = site_soup.find_all(['p', 'div', 'span'])
                site_text = "\n".join([para.get_text() for para in paragraphs if len(para.get_text()) > 30])

            content_dict[site] = site_text[:200000]
            time.sleep(2)  # Attendre 2 secondes entre chaque requête pour éviter un blocage

        except Exception as e:
            print(f"Erreur lors de la récupération du contenu de {site}: {e}")
            continue
    
    return content_dict




In [35]:
def get_websites_from_questions(questions_list, k):
    websites_list = []
    for q in tqdm(questions_list):
        websites_list.append(google_search_scraper(q, k))
    return websites_list

google_queries = pd.read_csv('google_queries.csv')

100%|██████████| 2/2 [00:25<00:00, 12.96s/it]


In [41]:
google_queries = pd.read_csv('google_queries.csv')

In [58]:
websites_list = get_websites_from_questions(questions_list=google_queries['Query'], k=3)

  1%|          | 1/103 [00:12<21:34, 12.69s/it]

Erreur lors de la récupération du contenu de https://fedecardio.org/je-m-informe/l-insuffisance-cardiaque/: HTTPSConnectionPool(host='fedecardio.org', port=443): Max retries exceeded with url: /je-m-informe/l-insuffisance-cardiaque/ (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1133)')))


 11%|█         | 11/103 [01:43<14:48,  9.66s/it]

Erreur lors de la récupération du site https://www.sciencedirect.com/science/article/abs/pii/S1169833007000117 (code 403)


 12%|█▏        | 12/103 [01:49<13:03,  8.61s/it]

...

 83%|████████▎ | 85/103 [12:00<02:06,  7.04s/it]

Erreur lors de la récupération du contenu de https://sofia.medicalistes.fr/spip/IMG/pdf/59_etat_d_agitation_psychomotrice.pdf: HTTPSConnectionPool(host='sofia.medicalistes.fr', port=443): Max retries exceeded with url: /spip/IMG/pdf/59_etat_d_agitation_psychomotrice.pdf (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1133)')))


 90%|█████████ | 93/103 [13:07<01:24,  8.40s/it]

Erreur lors de la récupération du site https://cuen.fr/manuel3/spip.php?article119 (code 404)


100%|██████████| 103/103 [14:47<00:00,  8.62s/it]


In [78]:
websites_df.to_csv("websites.csv", index=False)


In [76]:
websites_df = []

# Iterate over the list of dictionaries with their indices
for index, dictionary in enumerate(websites_list):
    for key, value in dictionary.items():
        websites_df.append({"Index": index, "Url": key, "Text": value})

# Convert the transformed data into a pandas DataFrame
websites_df = pd.DataFrame(websites_df)

In [77]:
websites_df[websites_df['Index']==0]

,Index,Url,Text
0,0,https://www.medg.fr/exantheme-chez-ladulte-et-...,Exanthème de l'enfant - MedG\nLe projet\nDétai...
1,0,https://www.hug.ch/sites/interhug/files/struct...,%PDF-1.3\n%���������\n4 0 obj\n<< /Length 5 0 ...
2,0,https://pap-pediatrie.fr/dermatologie/eruption...,Éruption cutanée fébrile de l’enfant | Pas à P...


In [57]:
for key, value in websites_list[].items():
    print(len(value),key)


20934 https://www.msdmanuals.com/fr/professional/h%C3%A9matologie-et-oncologie/leuc%C3%A9mies/leuc%C3%A9mie-lympho%C3%AFde-chronique-llc
200000 https://www.e-cancer.fr/content/download/465921/7056870/file/Guide-MG-Leuc%C3%A9mie%20lympho%C3%AFde%20chronique%20-%20Du%20diagnostic%20au%20suivi_2023.pdf
10379 https://www.msdmanuals.com/fr/accueil/troubles-du-sang/leuc%C3%A9mies/leuc%C3%A9mie-lympho%C3%AFde-chronique-llc


In [66]:
web = pd.read_csv('websites.csv')
web

,https://www.medg.fr/exantheme-chez-ladulte-et-lenfant/,https://www.hug.ch/sites/interhug/files/structures/saup_professionnels/fichiers/erruptions_cutanees_st_justine.pdf,https://pap-pediatrie.fr/dermatologie/eruption-cutanee-febrile-de-lenfant,https://www.capretraite.fr/blog/sante/insuffisance-cardiaque-les-7-fausses-croyances/,https://www.has-sante.fr/jcms/c_1242983/fr/guide-parcours-de-soins-de-l-insuffisance-cardiaque,https://www.msdmanuals.com/fr/accueil/troubles-pulmonaires-et-des-voies-a%C3%A9riennes/bronchectasie-et-at%C3%A9lectasie/bronchectasie,https://www.revmed.ch/revue-medicale-suisse/2004/revue-medicale-suisse-2502/bronchectasies-diagnostic-et-prise-en-charge,https://www.msdmanuals.com/fr/professional/troubles-pulmonaires/bronchectasies-et-at%C3%A9lectasies/bronchectasies,https://www.has-sante.fr/jcms/c_1319266/fr/argumentaire-antiagregants-plaquettaires,https://www.realites-cardiologiques.com/wp-content/uploads/sites/2/2018/10/Delahaye_Double_antiagregation_dans_coronaropathies.pdf,...,https://www.msdmanuals.com/fr/accueil/la-sant%C3%A9-des-personnes-%C3%A2g%C3%A9es/le-vieillissement/changements-corporels-du-vieillissement,https://www.has-sante.fr/upload/docs/application/pdf/2013-06/fiche_parcours_fragilite_vf.pdf,https://www.nutricia.fr/aires-therapeutiques/fragilite/,http://www.gerontopolesud.fr/d%C3%A9tection-de-la-fragilit%C3%A9/d%C3%A9finitions,https://www.sfmu.org/upload/consensus/rfe_tcl_sfmu2012.pdf,https://www.has-sante.fr/upload/docs/application/pdf/2008-09/surdosage_en_avk_situations_a_risque_et_accidents_hemorragiques_-_synthese_des_recommandations_v2.pdf,https://www.sfmu.org/upload/70_formation/02_eformation/02_congres/Urgences/urgences2010/donnees/pdf/055_tazarourte.pdf,https://www.has-sante.fr/jcms/p_3297884/fr/diagnostic-de-la-denutrition-chez-la-personne-de-70-ans-et-plus-recommandations,https://www.has-sante.fr/jcms/p_3297885/fr/diagnostic-de-la-denutrition-chez-l-enfant-l-adulte-et-la-personne-de-70-ans-et-plus-fiche-outil,https://recomedicales.fr/recommandations/denutrition/
0,Exanthème de l'enfant - MedG\nLe projet\nDétai...,%PDF-1.3\n%���������\n4 0 obj\n<< /Length 5 0 ...,Éruption cutanée fébrile de l’enfant | Pas à P...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,Insuffisance Cardiaque : 7 Fausses Croyances -...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,Bronchectasie - Bronchectasie - Manuels MSD po...,Bronchectasies : diagnostic et prise en charge...,Bronchectasies - Bronchectasies - Édition prof...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,%PDF-1.5\n%����\n222 0 obj\n<\n>\nendobj\nxref...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
99,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Changements corporels du vieillissement - Chan...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,%PDF-1.4\n%‚„œ”\n1242 0 obj\n<\n>\nendobj\nxre...,"Fragilité des personnes âgées - définition, si...",Définitions | Gérontopôle Sud\nAller au conten...,NaN,NaN,NaN,NaN,NaN,NaN
101,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,%PDF-1.3\n%����\n88 0 obj\n<\n>\nendobj\nxref\...,%PDF-1.4\n%�쏢\n5 0 obj\n<\n>\nstream\nx��}ݒ\��...,%PDF-1.3\n%����\n1 0 obj\n<<\n/Length 798\n>>\...,NaN,NaN,NaN


In [28]:
for key, value in websites_list[0].items():
    print(len(value))
    print(key)

4279
https://papillons.natagora.be/reconnaitre-les-papillons
95703
https://fr.wikipedia.org/wiki/Lepidoptera
14094268
https://www.sciences-participatives-au-jardin.org/files/Guide-papillons.pdf
5854
https://www.lepinet.fr/identifier/miniatures.php?e=
